# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_file="Output/weather_check.csv"
weather_check=pd.read_csv(weather_file)
weather_check


,City,Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country
0,Atuona,79.50,73,99,15.10,-9.8000,-139.0333,PF
1,Dikson,29.14,97,100,6.24,73.5069,80.5464,RU
2,Torbay,50.00,71,75,18.41,47.6666,-52.7314,CA
3,Hofn,42.80,49,20,19.57,64.2539,-15.2082,IS
4,Isangel,72.75,60,27,13.00,-19.5500,169.2667,VU
...,...,...,...,...,...,...,...,...
559,Merano,68.00,42,0,9.22,46.6682,11.1595,IT
560,Kem',34.14,77,100,4.52,64.9555,34.5793,RU
561,Nena,75.04,89,73,4.03,11.7658,125.4178,PH
562,Kulhudhuffushi,84.18,73,70,8.21,6.6221,73.0700,MV


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [24]:
locations = weather_check[["Latitude", "Longitude"]].astype(float)
weather_check=weather_check.dropna()
humidity=weather_check["Humidity"].astype(float)
len(locations)

561

In [26]:
fig = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False,
                                 max_intensity=150,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
#My favorite weather conditions would be 60-70 degrees F, cloudy and humid
hotel_df=weather_check.loc[(weather_check["Temp"]>60)&
                                (weather_check["Temp"]<70)&
                                (weather_check["Humidity"]>80)&
                                (weather_check["Cloudiness"]>80)]
hotel_df

,City,Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country
10,Hermanus,60.01,81,92,1.99,-34.4187,19.2345,ZA
42,Busselton,61.65,96,100,4.21,-33.6500,115.3333,AU
48,Bowen,68.00,94,90,5.75,-20.0167,148.2333,AU
54,Naze,67.42,83,100,14.20,28.3667,129.4833,JP
146,Xiongzhou,62.98,93,100,1.48,25.1167,114.3000,CN
160,Kapaa,67.59,88,90,4.61,22.0752,-159.3190,US
165,Pisco,66.11,82,90,9.22,-13.7000,-76.2167,PE
222,Rantepao,66.00,97,100,2.44,-2.9701,119.8978,ID
354,Anzoátegui,67.01,81,100,4.63,4.6300,-75.0954,CO
417,Shingū,62.38,95,100,10.29,33.7333,135.9833,JP


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df["Hotel Name"]=""
loc_lat=hotel_df.iloc[:,5].tolist()
loc_lon=hotel_df.iloc[:,6].tolist()



C:\Users\Josh\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
hotel_name=[]
coordinates=[]
for x in range(len(hotel_df)):
    base_url=f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={loc_lat[x]},{loc_lon[x]}&radius=5000&type=lodging&key={g_key}"
    response = requests.get(base_url).json()
    try:
            hotel_name.append(response["results"][0]["name"])
            coordinates.append((loc_lat[x],loc_lon[x]))
            print(response["results"][0]["name"]+" added")
    except:
            hotel_name.append("Hotel not found")
            print("Hotel not found")
    

Misty Waves Boutique Hotel added
Observatory Guest House added
Castle Motor Lodge Motel added
奄美ポートタワーホテル added
Xinhewan Business Hotel added
Sheraton Kauai Resort at Coconut Beach added
Hotel San Isidro Oficial added
Hotel Luta Resort added
Hotel Y Restaurante El Nido added
Shingu UI Hotel added
Amelia Guest House added


In [31]:
for x in range(len(hotel_name)):
    hotel_df.iloc[x,8] = hotel_name[x]
    
hotel_df


C:\Users\Josh\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country,Hotel Name
10,Hermanus,60.01,81,92,1.99,-34.4187,19.2345,ZA,Misty Waves Boutique Hotel
42,Busselton,61.65,96,100,4.21,-33.6500,115.3333,AU,Observatory Guest House
48,Bowen,68.00,94,90,5.75,-20.0167,148.2333,AU,Castle Motor Lodge Motel
54,Naze,67.42,83,100,14.20,28.3667,129.4833,JP,奄美ポートタワーホテル
146,Xiongzhou,62.98,93,100,1.48,25.1167,114.3000,CN,Xinhewan Business Hotel
160,Kapaa,67.59,88,90,4.61,22.0752,-159.3190,US,Sheraton Kauai Resort at Coconut Beach
165,Pisco,66.11,82,90,9.22,-13.7000,-76.2167,PE,Hotel San Isidro Oficial
222,Rantepao,66.00,97,100,2.44,-2.9701,119.8978,ID,Hotel Luta Resort
354,Anzoátegui,67.01,81,100,4.63,4.6300,-75.0954,CO,Hotel Y Restaurante El Nido
417,Shingū,62.38,95,100,10.29,33.7333,135.9833,JP,Shingu UI Hotel


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
Name: {Hotel Name}
City: {City}
Country: {Country}
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info

['\nName: Misty Waves Boutique Hotel\nCity: Hermanus\nCountry: ZA\n',
 '\nName: Observatory Guest House\nCity: Busselton\nCountry: AU\n',
 '\nName: Castle Motor Lodge Motel\nCity: Bowen\nCountry: AU\n',
 '\nName: 奄美ポートタワーホテル\nCity: Naze\nCountry: JP\n',
 '\nName: Xinhewan Business Hotel\nCity: Xiongzhou\nCountry: CN\n',
 '\nName: Sheraton Kauai Resort at Coconut Beach\nCity: Kapaa\nCountry: US\n',
 '\nName: Hotel San Isidro Oficial\nCity: Pisco\nCountry: PE\n',
 '\nName: Hotel Luta Resort\nCity: Rantepao\nCountry: ID\n',
 '\nName: Hotel Y Restaurante El Nido\nCity: Anzoátegui\nCountry: CO\n',
 '\nName: Shingu UI Hotel\nCity: Shingū\nCountry: JP\n',
 '\nName: Amelia Guest House\nCity: Moyo\nCountry: UG\n']

In [37]:
# Add marker layer ontop of heat map


figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,map_type="HYBRID")
markers = gmaps.marker_layer(coordinates,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig
# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…